In [ ]:
from nos.client import InferenceClient, TaskType

# Create a client that connects to the inference server via gRPC (50051)
client = InferenceClient()

In [ ]:
client.WaitForServer()

In [ ]:
client.IsHealthy()

In [ ]:
import requests
from PIL import Image


url = "https://raw.githubusercontent.com/open-mmlab/mmdetection/main/demo/demo.jpg"
img = Image.open(requests.get(url, stream=True).raw).resize((640, 480))
img

In [ ]:
import cv2
import numpy as np

def visualize_det2d(img: np.ndarray, bboxes: np.ndarray, labels: np.ndarray) -> np.ndarray:
    """Visualize 2D detection results on an image."""
    vis = np.asarray(img).copy()
    for bbox, label in zip(bboxes.astype(np.int32), labels):
        cv2.rectangle(vis, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 2)
    return vis

# Run YOLOX prediction on the image and get the prediction results as a dictionary.
# predictions = {"bboxes", "scores", "labels"}.
predictions = client.Run(TaskType.OBJECT_DETECTION_2D, "yolox/medium", images=[img])
for idx, (img, bboxes, scores, labels) in enumerate(zip([img], predictions["bboxes"], predictions["scores"], predictions["scores"])):
    display(Image.fromarray(visualize_det2d(img, bboxes, labels)))

In [ ]:
# Let's do the same for yolox/nano-trt (The TensorRT version of the above model). Note that this one will take a bit longer on the first pass since we need to compile it first.
predictions = client.Run(TaskType.OBJECT_DETECTION_2D, "yolox/medium-trt", images=[img])
for idx, (img, bboxes, scores, labels) in enumerate(zip([img], predictions["bboxes"], predictions["scores"], predictions["scores"])):
    display(Image.fromarray(visualize_det2d(img, bboxes, labels)))

In [ ]:
%%timeit -n 100
predictions = client.Run(TaskType.OBJECT_DETECTION_2D, "yolox/medium", images=[img])

In [ ]:
# Run a single prediction with TRT to trigger compilation so we don't skew timing results
predictions = client.Run(TaskType.OBJECT_DETECTION_2D, "yolox/medium-trt", images=[img]) 

In [ ]:
%%timeit -n 100
predictions = client.Run(TaskType.OBJECT_DETECTION_2D, "yolox/medium-trt", images=[img])